# GlaxoSmithKline - Accumulations

In [2]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Format Table

In [4]:
df_export = pd.DataFrame(columns=fix_columns_accumulations[:-1])

# Add HCP
df_export = add_accumulation(df_export, AccType.hcp_amount, 
                             registration_fees = '', 
                             travel_accommodation = '', 
                             fees = '', 
                             related_expenses = '')

df_export = add_accumulation(df_export, AccType.hcp_count, 
                             registration_fees = '', 
                             travel_accommodation = '', 
                             fees = '', 
                             related_expenses = 0)

# Add HCO
df_export = add_accumulation(df_export, AccType.hco_amount,
                            donations_grants = '',
                            sponsorship = '2400',
                            fees = '',
                            total = '2400')

df_export = add_accumulation(df_export, AccType.hco_count,
                            donations_grants = '',
                            sponsorship = '1',
                            fees = '',
                            total = '1')
# RnD
df_export = add_accumulation(df_export, AccType.rnd,
                            donations_grants = '854648.40')

#Numberize and sum
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Fix RnD
df_export.loc[df_export.type == 'rnd', 'donations_grants'] = np.NaN

export_acumulations(df_export, 'glaxosmithkline')

saved


In [16]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)